# Analytically solving the pressure wave equation

Here we use the method described by John Learned in his paper about *[Acoustic Radiation due to Charged Particles](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.19.3293)*. This method requires solving for the attenuation of a wave based on its original distribution, and then approximating the original distribution based on the unatenuated term.


Specifically, we initially find a solution to the following equation in fourier space. The pressure is given as a function $p(\vec{x},t)$ where: $p : \mathbb{R}^3 \times \mathbb{R}^+ \rightarrow \mathbb{R}$

$$
\Delta \left(p -\frac{1}{\omega_0} p_t \right) - \frac{1}{c^2} p_{tt} = 0
$$

Now we define the following fourier transform $\hat{p}(\vec{x},\omega)$ of $p(\vec{x},\omega)$.

$$
\hat{p}(\vec{x},\omega) = \int_{\mathbb{R}} p(\vec{x},t) e^{-i\omega t} dt
$$

Therefore applything the fourier transform to both sides of the equation we obtain:

$$
\left( 1 + i\frac{\omega}{\omega_0} \right) \Delta \hat{p}(\vec{x},\omega) + \frac{\omega^2}{c^2} \hat{p}(\vec{x},t) = 0
$$

Which we can reqrite as:

$$
\Delta \hat{p}(\vec{x},\omega) + k^2 \hat{p}(\vec{x},t) = 0
$$

where $k$ is:

$$ k = \pm \frac{\omega}{c^2} \frac{1}{\sqrt{1+i\frac{\omega}{\omega_0}}}$$

Now we can assume $\hat{p}(\vec{x},\omega) = \hat{p}(|\vec{x}|,\omega) = \hat{p}(r,\omega)$. With this we can introduce the parameterisation of $\hat{p}$ as $\hat{p}(r,\omega) = \hat{p}_r(r,\omega)/r$. Therefore we can reqrite our original equation now in spherical coordinates like so:

$$
\frac{1}{r} \frac{\partial^2 \hat{p}_r}{\partial r^2} + \frac{k^2}{r} \hat{p}_r = 0
$$

This is a very simple ODE wich we can solve by:

$$
\hat{p}(r,\omega) = \frac{1}{r} \hat{p}_0(\omega) e^{-i k r} 
$$

Now given the initial pulse ar around time-space 0 we can figure out how it evolves over time. To do that we solve the non-viscous wave equation for a particular heat distribution obtain $\hat{p}_0$ and then substitude it in the solution above. Doing so we will obtain that for a gaussian source of heat we get the following fourier spectrum for the pulse.

$$
\hat{p}(r,\omega) = i \frac{\beta E_0}{4\pi r C_p}\omega \exp{\left(i\frac{\omega r}{c} + \frac{\omega^2 r}{2 \omega_0 c} + \frac{\sigma^2 \omega^2}{2}\right)}
$$

Now we use fourier's identity to obtain the time dependent version of the equation.

$$
{p}(r,t) = - \frac{\beta E_0}{4\pi r C_p} \frac{ct - r}{\sqrt{2\pi c^2} \left(\sigma^2 + \frac{r}{\omega_0 c}\right)^{\frac{3}{2}} }  \exp{\left(- \frac{(ct - r)^2}{2c^2 \sigma^2 + \frac{rc}{\omega_0}}\right)}
$$

Now let's plot!

In [26]:
#import relevant libraries
import numpy as np
import scipy.constants as const
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from tqdm import tqdm

# if this doesn't work run: "python3 -m pip install ipympl" 
# or just comment it out and learn to live without sliders on the graphs
%matplotlib widget


# Define the relevant constants in SI units
beta    = 4.65e-3
E_0     = 2*1e6*const.eV
C_p     = 338.48
c       = 653.47
w_0     = 1.2667e+9/1.70e-2
sigma   = (2.1801714e-25/2966.3)**(1/3)
rho     = 2966.3
T       = 161.36

# Units
m   = 1
mm  = 1e+3
s   = 1
ms  = 1e+3
us  = 1e+6


# Function for the radial pressure wave due to a point source
def p(r, t, E_0=E_0, beta=beta, C_p=C_p, c=c, w_0=w_0, sigma=sigma):
    if r < 0:
        r = - r
    
    if r == 0:
        return 0
    
    return -beta*E_0*(c*t - r)/(4*np.pi*C_p* (2*np.pi*c**2)**0.5 * (sigma**2 + r/(w_0*c)))* np.exp(-(c*t-r)**2/(2*c**2*sigma**2+(r*c)/(w_0)))


def signal_noise(r,beta=beta,c=c,E_0=E_0,C_p=C_p,rho=rho,T=T,sigma=sigma,w_0=w_0):
    if r==0:
        return 0

    if r < 0:
        r = -r

    return beta**2 * c * E_0**2/(16*np.pi**(3/2)*C_p**2*rho*const.k*T*r**2*(sigma**2 + r/(w_0*c))**(1/2))

In [31]:
# Plot the point source sound wave as a function of spacetime

# Set up the plot
fig = plt.figure(figsize=(9,10))
fig.suptitle("Pressure due to point source in Liquid Xenon")
ax = fig.add_subplot(211)
ax.set_title('Pressure VS Radial Distance')

r_unit = mm
t_unit = us

r_min = 0
r_max = 1e-3
t_min = 0
t_max = abs(r_max-r_min)/c
Npts = 1000

ax.set_xlim([r_min*r_unit, r_max*r_unit])
ax.set_ylabel(r'$p(r,t)\ [Pa]$',fontsize = 15)
ax.set_xlabel(r'$r\ [mm]$',fontsize = 15)
ax.grid(True)

# Generate x-y values
R = np.linspace(r_min,r_max,Npts)
def p_vect(R, t, E_0=E_0, beta=beta, C_p=C_p, c=c, w_0=w_0, sigma=sigma):
    return np.array([p(r, t, E_0=E_0, beta=beta, C_p=C_p, c=c, w_0=w_0, sigma=sigma) for r in R])

# Add the sliders
scale = 1e4
@widgets.interact(t=(t_min*scale, t_max*scale, abs(t_min-t_max)*1e-4*scale), Npts=(100,10000, 1))
def update(t=(t_max-t_min)/2, Npts=1000):
    [l.remove() for l in ax.lines]
    R = np.linspace(r_min,r_max,Npts)
    P = p_vect(R,t/scale)
    print(R[np.where(P==max(P))],R[np.where(P==min(P))],' DIST: ',R[np.where(P==min(P))]-R[np.where(P==max(P))])
    ax.plot(R*r_unit, P, color='C3',label=r'$t = %3.2e\ s$ '%(t/scale)+"\n"+'$p_{max} = %3.2e\ Pa$'%(max(P)))
    ax.legend()

# Add the plot of Maximum Power over time
ax2 = fig.add_subplot(212)
ax2.set_xlim([t_min*t_unit, t_max*t_unit])
ax2.set_ylabel(r'$p(r,t)\ [Pa]$',fontsize = 15)
ax2.set_xlabel(r'$t\ [\mu s]$',fontsize = 15)
ax2.grid(True)

Tt = np.linspace(t_min,t_max,int(1e+2))
R = np.linspace(r_min,r_max,int(8e+3))
P_t = [max(p_vect(R,t)) for t in tqdm(Tt)]
ax2.plot(Tt*t_unit, P_t, color='C9')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=7.651460663840727e-07, description='t', max=0.015302921327681454, step…

100%|██████████| 100/100 [00:06<00:00, 15.66it/s]


# Signal to Noise

According to Learned we can get the signal to noise ratio against thermal noise for any fluid. This can begicen by the following relation.

$$
\frac{S}{N} = \frac{\beta^2 c}{16 \pi^{\frac{3}{2}} C_p^2 \rho k T} \frac{E_0^2}{r^2 \left(\sigma^2 +\frac{r}{\omega_0 c} \right)^{\frac{1}{2}}}
$$

We can plot this against distance to get an estimate of such a ratio

In [30]:
# Plot the point source sound wave as a function of spacetime

# Set up the plot
fig = plt.figure(figsize=(9,6))
fig.suptitle("Pressure due to point source in Liquid Xenon")
ax = fig.add_subplot(111)
ax.set_title('Signal to Noise VS Radial Distance')

r_unit = mm
t_unit = us

r_min = 5e-5
r_max = 1e-3
Npts  = int(8e+3)

ax.set_xlim([r_min*r_unit, r_max*r_unit])
ax.set_ylabel(r'$\frac{S}{N}(r)$',fontsize = 15)
ax.set_xlabel(r'$r\ [mm]$',fontsize = 15)
ax.grid(True)

# Generate x-y values
def signal_noise_vect(R ,beta=beta,c=c,E_0=E_0,C_p=C_p,rho=rho,T=T,sigma=sigma,w_0=w_0):
    return np.array([signal_noise(r,beta=beta,c=c,E_0=E_0,C_p=C_p,rho=rho,T=T,sigma=sigma,w_0=w_0) for r in R])

# Add the plot of Maximum Power over time

R = np.linspace(r_min,r_max,Npts)
SN = signal_noise_vect(R)
ax.plot(R*r_unit, SN, color='C10')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …